Привет, меня зовут Артем. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* В работе я оставил несколько советов. Буду рад, если ты учтешь их.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Привет, Артем! Спасибо за комментарии. Поправила и добавила комментарии про изменения
</div>

### <font color='orange'>Общее впечатление (ревью 2)</font>
* Для удобства все новые комментарии обозначены фразой "ревью 2".
* Удачи в дорабокте!

### <font color='orange'>Общее впечатление (ревью 3)</font>
* После исправлений проект улучшился и теперь он может быть зачтен.
* Удачи в дальнейшем обучении и следующих работах!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
!pip install phik ydata_profiling shap numpy==1.23 numba==0.58.1 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 2.8.2 which is incompatible.
spacy 3.2.0 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 2.8.2 which is incompatible.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import phik
import re
import scipy.stats as st
import seaborn as sns
import statsmodels.api as sm
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import shap
import ydata_profiling

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

<b> Описание данных: </b> 
  Данные геологоразведки трёх регионов находятся в файлах: 
`/datasets/geo_data_0.csv.` 
`/datasets/geo_data_1.csv.` 
`/datasets/geo_data_2.csv.` 

`id` — уникальный идентификатор скважины;  
`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);  
`product` — объём запасов в скважине (тыс. баррелей).

In [3]:
# Считаем CSV-файлы с данными с помощью библиотеки pandas в датафрейм и изучим общую информацию

path = '/datasets'

try:
    # Чтение датафреймов
    geo_data = pd.read_csv(os.path.join(path, 'geo_data_0.csv'), index_col='id')
    geo_data1 = pd.read_csv(os.path.join(path, 'geo_data_1.csv'), index_col='id')
    geo_data2 = pd.read_csv(os.path.join(path, 'geo_data_2.csv'), index_col='id')

except FileNotFoundError:
    print('Один или несколько файлов не найдены.')

In [4]:
geo_data.sample(3)

,f0,f1,f2,product
id,,,,
FgLK4,0.722486,0.156214,3.743333,136.940110
GQbdb,0.065091,1.059783,7.337488,101.731406
Qnd7y,0.992474,0.537133,-2.278604,49.876440


In [5]:
geo_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [6]:
geo_data1.sample(3)

,f0,f1,f2,product
id,,,,
kAEny,-4.341771,-4.536024,4.005632,110.992147
7LQc0,2.390416,-10.812846,2.009577,53.906522
Giq0D,-7.349398,0.276799,1.001946,30.132364


In [7]:
geo_data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [8]:
geo_data2.sample(3)

,f0,f1,f2,product
id,,,,
eYFSu,-0.209147,-0.932120,1.788196,36.537653
J1fc0,-0.647266,1.455445,-0.800666,65.410047
flAww,1.203650,-0.317457,7.409175,118.019639


In [9]:
geo_data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [10]:
geo_data = geo_data.sort_index()
geo_data1 = geo_data1.sort_index()
geo_data2 = geo_data2.sort_index()

<b>По результатам загрузки</b> видим, что полученные данные соответствуют их описанию.

In [11]:
# ydata_profiling.ProfileReport(geo_data)

<b>В первом регионе:</b>
- пропущенных значений нет,
- необычных значений признаков нет,
- f0 в диапазоне от -1.4 до 2.3,
- f1 в диапазоне от -0.8 до 1.3,
- f2 в диапазоне от -12 до 16,
- product от 0 до 185,
- интересный график корреляции между f0 и f1 - две зеркальные дуги,
- похожа на линейную зависимость между product и f2, 
- предварительно на матрице корреляции большая зависимость между f0 и f1, product и f2

In [12]:
# ydata_profiling.ProfileReport(geo_data1)

<b>Во втором регионе:</b>
- нет пропусков,
- тоже похожа на линейную зависимость между product и f2 (почти 1 на матрице),
- значительная зависимость между f0 и f1

In [13]:
# ydata_profiling.ProfileReport(geo_data2)

<b>В третьем регионе:</b>
- нет пропусков,
- также высокая зависимость между f2 и product

In [14]:
# удалим дубликаты:
geo_data.drop_duplicates(inplace=True)
geo_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 006OJ to zzzLH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [15]:
# Найдем дублирующиеся индексы
duplicate_indexes = geo_data.index[geo_data.index.duplicated()]
duplicate_indexes

Index(['74z30', 'A5aEY', 'AGS9W', 'HZww2', 'QcMuo', 'Tdehs', 'TtcGQ', 'bsk9y',
       'bxg6G', 'fiKDv'],
      dtype='object', name='id')

In [16]:
# Добавляем порядковый номер к дублирующимся индексам
for index in duplicate_indexes:
    count = 1
    for i in range(len(geo_data)):
        if geo_data.index[i] == index:
            geo_data.index.values[i] = index + str(count)
            count += 1

geo_data.index[geo_data.index.duplicated()]

Index([], dtype='object', name='id')

In [17]:
geo_data1.drop_duplicates(inplace=True)
geo_data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0022J to zzzvI
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [18]:
duplicate_indexes1 = geo_data1.index[geo_data1.index.duplicated()]
duplicate_indexes1

Index(['5ltQ6', 'LHZR0', 'bfPNe', 'wt4Uk'], dtype='object', name='id')

In [19]:
for index in duplicate_indexes1:
    count = 1
    for i in range(len(geo_data1)):
        if geo_data1.index[i] == index:
            geo_data1.index.values[i] = index + str(count)
            count += 1

geo_data1.index[geo_data1.index.duplicated()]

Index([], dtype='object', name='id')

In [20]:
geo_data2.drop_duplicates(inplace=True)
geo_data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 009Gl to zzz9h
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [21]:
duplicate_indexes2 = geo_data2.index[geo_data2.index.duplicated()]
duplicate_indexes2

Index(['KUPhW', 'VF7Jo', 'Vcm5J', 'xCHr8'], dtype='object', name='id')

In [22]:
# Добавляем порядковый номер к дублирующимся индексам
for index in duplicate_indexes2:
    count = 1
    for i in range(len(geo_data2)):
        if geo_data2.index[i] == index:
            geo_data2.index.values[i] = index + str(count)
            count += 1

geo_data2.index[geo_data2.index.duplicated()]

Index([], dtype='object', name='id')

<b>По результатам анализа и обработки данных:</b>
- проверили дубликаты и пропуски,
- проанализировали распределение признаков и корреляции между входными признаками для будущих моделей.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены и осмотрены корректно. Радует, что данные были визуализированы!
</div>

## Обучение и проверка модели

Обучим модель для каждого региона:
- данные разобьем на обучающую и валидационную выборки в соотношении 75:25.
- обучите модель линейной регрессии и сделаем предсказания на валидационной выборке.
- сохраним предсказания и правильные ответы на валидационной выборке.
- напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [23]:
dataframes = [geo_data, geo_data1, geo_data2]
target = 'product'
num_columns = ['f0', 'f1', 'f2']

# пайплайны для обработки данных
missing_values_pipe = Pipeline([
    ('simpleImputer_missing', SimpleImputer(strategy='most_frequent'))
])

ohe_pipe = Pipeline([
    ('missing_values', missing_values_pipe),
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
])


data_preprocessor = ColumnTransformer([
    ('num', Pipeline([('missing_values', missing_values_pipe), ('scaler', StandardScaler())]), num_columns)
], remainder='passthrough')

# пайплайн для модели
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('model', LinearRegression())
])

param_grid_lr = {
    'model__fit_intercept': [True, False],
    'model__normalize': [True, False],
    'model__copy_X': [True, False],
    'model__positive': [True, False]
}

In [24]:
# Создаем список для хранения моделей и датафреймов с предсказаниями
models_list = []
dataframes_with_predictions = []

# Цикл для обучения и проверки модели для каждого датафрейма
for i, df in enumerate(dataframes):
    print(f"* Регион {i+1} *")
    
    # Разделите данные на обучающую и валидационную выборки
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Обучение модели
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Сохранение модели
    models_list.append(model)
    
    # Предсказания на валидационной выборке
    y_pred = model.predict(X_valid)
    
    # Сохранение предсказаний и правильных ответов в отдельный датафрейм
    region_predictions = pd.DataFrame({
        'true_value': y_valid,
        'predicted_value': y_pred
    })
    
    # Добавление датафрейма с предсказаниями и правильными ответами в список
    dataframes_with_predictions.append(region_predictions)
    
    # Вывод на экран среднего запаса предсказанного сырья и RMSE модели
    print(f"Средний запас предсказанного сырья: {y_pred.mean():.2f}")
    print(f"RMSE: {mean_squared_error(y_valid, y_pred, squared=False):.2f}")    
    print("-" * 50)

* Регион 1 *
Средний запас предсказанного сырья: 92.34
RMSE: 37.75
--------------------------------------------------
* Регион 2 *
Средний запас предсказанного сырья: 68.70
RMSE: 0.89
--------------------------------------------------
* Регион 3 *
Средний запас предсказанного сырья: 95.01
RMSE: 40.08
--------------------------------------------------


In [25]:
dataframes_with_predictions

[       true_value  predicted_value
 id                                
 kz06D   70.787822        89.912467
 nmARb   78.120803        78.870332
 CFf4y   91.914529        79.366452
 lbPfa   50.874167        90.235009
 viqqq   29.274970        58.324255
 ...           ...              ...
 D7FRV   57.225917        78.004614
 L1nLN   99.970187       110.653576
 oTy7k   32.224135        57.860580
 ekvm5  139.008774        88.476621
 7B93G   59.209991        59.688020
 
 [25000 rows x 2 columns],
        true_value  predicted_value
 id                                
 ksHYQ    3.179103         4.102905
 ngLE2  134.766305       136.002882
 CIj94   53.906522        53.526653
 lVc5R   57.085625        56.213575
 vfVtG    3.179103         2.498790
 ...           ...              ...
 DAgbN   30.132364        28.071428
 Ky4ay   57.085625        55.484537
 oQwFz   30.132364        29.101843
 eeopb   30.132364        29.928177
 7FsJv    3.179103         2.673879
 
 [25000 rows x 2 columns],
      

<b>Построили модель и по результатам: </b>

- Регион 1
    - Средний запас предсказанного сырья: 92.34
    - RMSE: 37.75
- Регион 2
    - Средний запас предсказанного сырья: 68.70
    - RMSE: 0.89
- Регион 3
    - Средний запас предсказанного сырья: 95.01
    - RMSE: 40.08

Исходя из этих результатов, регион 2 выглядит наиболее перспективным для бурения новой скважины, так как модель для этого региона дает наиболее точные предсказания и показывает самый высокий средний предсказанный запас сырья.

<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно. Радует, что ты активно используешь циклы. Это помогает избежать дублирования кода, а значит уменьшает вероятность совершить ошибку/опечатку!
</div>

## Подготовка к расчёту прибыли

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. 
- Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [26]:
# Все ключевые значения для расчётов сохраним в отдельных переменных.

WELLS_IN_REGION = 500  # Количество скважин в регионе для исследования
WELLS_ML = 200  # Количество скважин в регионе для исследования после ML предсказаний
BUDGET = 10000000  # Бюджет на разработку (в тыс руб)

BARREL_PRICE = 450  # Доход с одного барреля (в рублях)
PRODUCT_PROFIT = 450  # Доход с каждой единицы продукта (в тыс рублей)

In [27]:
# Рассчитываем достаточный объём сырья для безубыточной разработки новой скважины

breakeven_reserve_well = BUDGET / PRODUCT_PROFIT / WELLS_ML
round(breakeven_reserve_well, 2)

111.11

<b> По результатам получили что: </b>
- 44,44 тыс баррелей - объем сырья, необходимый для безубыточной разработки, 
- 92 тыс баррелей - средний объем сырья в регионе 1, 
- 68 тыс баррелей - средний объем сырья в регионе 2, 
- 95 тыс баррелей - средний объем сырья в регионе 3. 

<div class="alert alert-block alert-info">

<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами: https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#id29 .
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Точка безубыточности найдена некорректно, так как разрабатывается 200 скважин, а не 500.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Обновила названия констант и расчет точки безубыточности
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка (ревью 2):</b> Я не вижу изменений.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения2:</b> Обновила формулу
</div>

## Расчёт прибыли и рисков 

Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберем скважины с максимальными значениями предсказаний. 
- Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитаем прибыль для полученного объёма сырья.

In [28]:
# Функция для выбора 200 лучших скважин и расчета прибыли
def calculate_profit(true_values, predicted_values, top_wells=200):
    # Создаем DataFrame из фактических и предсказанных значений
    df = pd.DataFrame({'true_value': true_values, 'predicted_value': predicted_values})
    
    # Сортируем DataFrame по предсказанному объему по убыванию
    df_sorted = df.sort_values(by='predicted_value', ascending=False)
    
    # Выбираем лучшие скважины по предсказанному объему
    top_df = df_sorted.head(top_wells)
    
    # Считаем прибыль по фактическому объему выбранных скважин
    profit = (top_df['true_value'].sum() * BARREL_PRICE - BUDGET) / 1000000  # делим на 1000000 чтобы получить результат в млрд руб
    return profit

Посчитаем риски и прибыль для каждого региона:
- Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [29]:
# Функция для bootstrap
def bootstrap_analysis(true_values, predicted_values, n=500, iterations=1000):
    state = np.random.RandomState(12345)
    profits = []

    for _ in range(iterations):
        # Выбираем случайную выборку из 500 скважин
        random_indices = state.choice(len(true_values), size=n, replace=True)
        
        # Выбираем фактические и предсказанные значения для выбранных скважин
        sampled_true_values = true_values.iloc[random_indices]
        sampled_predicted_values = predicted_values.iloc[random_indices]
        
        # Рассчитываем прибыль для этой выборки
        profit_value = calculate_profit(sampled_true_values, sampled_predicted_values)
        profits.append(profit_value)

    profits = pd.Series(profits)

    # Вычисляем среднюю прибыль
    mean_profit = profits.mean()
    
    # Вычисляем 95% доверительный интервал
    lower = profits.quantile(0.025)
    upper = profits.quantile(0.975)
    
    # Вычисляем риск убытка
    loss_probability = (profits < 0).mean()

    print('Средняя прибыль в регионе равна: %.2f' % (mean_profit), 'млн. руб.')
    print('95% доверительный интервал лежит в диапазоне: {:.2f}'.format(lower),
          '- {:.2f}'.format(upper))
    print('Вероятность убытка равна: {:.2%}'.format(loss_probability))

In [30]:
# Проводим bootstrap для каждого региона
for i, df in enumerate(dataframes_with_predictions):
    print(f"* Регион {i+1} *")
    true_values = df['true_value']
    predicted_values = df['predicted_value']
    bootstrap_analysis(true_values, predicted_values)
    print("-" * 50)

* Регион 1 *
Средняя прибыль в регионе равна: 0.46 млн. руб.
95% доверительный интервал лежит в диапазоне: -0.05 - 0.92
Вероятность убытка равна: 3.70%
--------------------------------------------------
* Регион 2 *
Средняя прибыль в регионе равна: 0.45 млн. руб.
95% доверительный интервал лежит в диапазоне: 0.05 - 0.84
Вероятность убытка равна: 1.50%
--------------------------------------------------
* Регион 3 *
Средняя прибыль в регионе равна: 0.38 млн. руб.
95% доверительный интервал лежит в диапазоне: -0.14 - 0.88
Вероятность убытка равна: 7.20%
--------------------------------------------------


<div class="alert alert-block alert-danger">
<b>Ошибка (ревью 2):</b> На каждой итерации бутстрапа нужно выбирать случайные 500 скважин из 25000, а не из 500.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения2:</b> Обновила расчет
</div>

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что для подсчета доверительного интервала использовался именно метод квантилей!
</div>


<div class="alert alert-block alert-info">

<b>Совет: </b> Риск можно было найти чуть проще: `(values < 0).mean()`, если объект типа pandas Series или numpy array.
</div>
    
<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Этот шаг нужно будет переработать.
    
1. В предыдущих шагах было получено 25000 настоящих и предсказанных значений для каждого региона.
2. Здесь нужно определить функцию, которая на вход будет принимать 500 настоящих и 500 предсказанных значений для каждого региона. Для выбора 200 лучших скважин нужно использовать предсказанные значения. А вот для подсчета прибыли должны использоваться настоящие значения соответствующих ячеек.
3. В следующем шаге нужно семплить 500 случайных скважин и передавать их в функцию для подсчета прибыли.
4. В результате у тебя получится 1000 значений прибыли для каждого региона. По нему нужно посчитать 3 статистики:
    * Среднюю прибыль
    * Интервал
    * Риск убытков
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Обновила расчет прибыли, bootstrap и расчет риска. И обновила вывод, тк лучший регион поменялся.
</div>

<b> По результатам анализа: </b>
Наилучшим выбором для разработки скважин является регион 1. В этом регионе средняя прибыль выше, чем в других регионах, при этом вероятность убытков составляет 0.5%.

## Общий вывод

В данной задаче мы провели анализ трёх регионов для определения наиболее перспективного места для бурения новой скважины с целью максимизации прибыли. 

1. Мы построили модели для предсказания объёма запасов нефти в новых скважинах в каждом из трёх регионов.
2. Оценили качество моделей по среднему запасу предсказанного сырья и RMSE.
3. Провели анализ возможной прибыли и рисков с использованием техники Bootstrap для каждого региона.
4. На основе полученных результатов, рекомендуем разрабатывать скважины в регионе 1, так как в этом регионе средняя прибыль выше, чем в других регионах и риск убытков ниже.

Таким образом, выбор региона для разработки новых скважин основан на анализе данных, построении моделей прогнозирования и оценке рисков. Рекомендация в пользу региона 1 обоснована его потенциально высокой прибылью.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован